In [32]:
from loaders import *

In [33]:
_loader = Loader(500)
loader = MIDILoader(_loader)

In [34]:
_sequence = loader.read('../data/bach_846.mid')
sequence = loader.tokenize(_sequence)

In [65]:
from collections import Counter
from gensim.models import Word2Vec
from random import random
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from torch import nn
from torch.autograd import Variable

import numpy as np
import torch
import torch.nn.functional as F
import json
import numpy as np
import matplotlib.pyplot as plt
import random

use_cuda = torch.cuda.is_available()
# Is the tokenizer 1 indexed?
vocabulary_size = 16*128*2 + 32*16 + 100 + 1 # 4708 + 1
encoding_size = 1000
one_hot_embeddings = np.eye(vocabulary_size) 

In [36]:
midi_sequence = loader.detokenize(np.array(sequence))
midi_sequence.write('test.mid')

In [56]:
class EncoderLSTM(nn.Module):
    # Your code goes here
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size).double()
        if use_cuda:
            self.lstm = self.lstm.cuda()
        
    def forward(self, input, hidden_in):
        _, hidden_out = self.lstm(input, hidden_in) # encoder only outputs hidden
        return hidden_out
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size)).double()
        if use_cuda:
            result = result.cuda()
        return result

In [60]:
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size).double()
        self.out = nn.Linear(hidden_size, output_size).double()
        self.project = nn.Linear(4096, self.hidden_size).double()

    def forward(self, input, hidden):
        output = F.relu(input)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = output.squeeze()
        return output.unsqueeze(0), hidden

    def initHidden(self, init_size, image_features):
        result = self.project(image_features)
        result = F.relu(result)
        if use_cuda:
            return result.cuda()
        else:
            return result

In [74]:
def train(sequence,
          encoder, 
          decoder, 
          encoder_optimizer, 
          decoder_optimizer, 
          criterion,
          embeddings=one_hot_embeddings):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # target_variable has (batch_size, length, vocab_size)
    # Without minibatch, this is just one sequence
    sequence_length = sequence.size()[1]
    loss = 0
    
    # Encoder is fed from the flipped sentence
    encoder_hidden = encoder.initHidden()
    encoder_hidden = (encoder_hidden, encoder_hidden) # Need a tuple
    
    # Feeding encoder in a loop, in reverse order
    for index_control in np.arange(sequence_length-1, -1, -1):
        encoder_input = sequence[0][index_control].view(1, 1, vocabulary_size)
        encoder_hidden = encoder(encoder_input, encoder_hidden) # Gets hidden for next input    
    
    # feed encoder_hidden
    decoder_input = sequence[0][0]
    decoder_hidden = encoder_hidden
    predicted_note_index = 0
    
    for index_control in range(1, sequence_length):
        decoder_input = decoder_input.view(1, 1, vocabulary_size)
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.data.topk(1)
        predicted_control_index = int(topi)

        
        if random.random() <= 0.9:
            decoder_input = sequence[0][index_control].view(1, 1, vocabulary_size)
        else:
            # This is the next input, without teacher forcing it's the predicted output
            decoder_input = torch.from_numpy(embeddings[predicted_control_index])
            decoder_input = Variable(decoder_input)
            if use_cuda:
                decoder_input = decoder_input.cuda()
        
        # This is just to conform with the pytorch format..
        # CrossEntropyLoss takes input1: (N, C) and input2: (N).
        _, actual_control_index = sequence[0][index_control].topk(1)
        if use_cuda:
            actual_control_index = actual_control_index.cuda()

        # Compare current output to next "target" input
        loss += criterion(decoder_output, actual_control_index)
            
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    # index_word keeps track of the current word
    # in case of break (EOS) and non-break (teacher-forcing), it'll be the actually count.
    return loss.data[0] / index_control

In [61]:
_loader = Loader(500)
loader = MIDILoader(_loader)

# Initialize the encoder with a hidden size of 1000. 
# With one-hot, the input size is
encoder = EncoderLSTM(vocabulary_size, encoding_size)
decoder = DecoderLSTM(vocabulary_size, encoding_size, vocabulary_size)

In [24]:
input_files = ['bach_846.mid', 'mz_311_1.mid', 'rac_op3_2.mid']

for input_file in input_files:
    sequence = loader.read('../data/' + input_file)
    sequence = loader.tokenize(sequence)
    print(min(sequence))

164
163
153


In [66]:
use_cuda = False

In [75]:
input_files = ['bach_846.mid', 'mz_311_1.mid', 'rac_op3_2.mid']
encoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1):
    for index, input_file in enumerate(input_files):
        sequence = loader.read('../data/' + input_file)
        sequence = loader.tokenize(sequence)
        seq_length = len(sequence)
        sequence = torch.from_numpy(np.array(one_hot_embeddings[sequence])) # This is really time consuming
        sequence = sequence.view(1, seq_length, vocabulary_size)
        sequence = Variable(sequence)
        if use_cuda:
            sequence = sequence.cuda()
        train(sequence, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        break
        
        

Variable containing:
1.00000e-02 *
-0.2433 -3.4444 -1.4732  ...   2.3321 -1.3483 -1.0829
[torch.DoubleTensor of size 1x4709]

Variable containing:
 188
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3443 -3.2816 -0.9515  ...   2.2017 -1.2624 -1.3304
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4628
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2621 -3.8619 -0.8256  ...   2.1845 -0.8001 -1.2733
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4143
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2294 -3.9790 -0.7408  ...   2.2218 -0.8398 -0.9466
[torch.DoubleTensor of size 1x4709]

Variable containing:
 192
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2246 -3.8437 -0.7879  ...   2.1980 -0.6979 -1.0245
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4629
[torch.LongTensor of size 1]

Variable containing:
-3.4768e-04 -3.9584e-02 -9.4381e-03  ...   2.0844e-02 -5.2453e-03 

Variable containing:
-8.0682e-04 -3.1751e-02 -1.2851e-02  ...   1.9759e-02 -8.6526e-03 -8.5559e-03
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4626
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.2334 -3.7967 -1.2062  ...   2.2103 -0.6212 -1.1667
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2252
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.5207 -3.3876 -1.3663  ...   2.0023 -0.7945 -1.6081
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4610
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.3028 -3.7173 -0.6425  ...   1.7943 -0.7208 -1.1836
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4141
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.0003 -3.6628 -0.7138  ...   2.2908 -0.2131 -0.8697
[torch.DoubleTensor of size 1x4709]

Variable containing:
 195
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.0878 -3.1049 -0.7534  ...   1.7149 -1.

Variable containing:
1.00000e-02 *
-0.0951 -3.3576 -0.7998  ...   2.0567 -0.8962 -0.9585
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4145
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3290 -3.4091 -1.0416  ...   1.7738 -0.7516 -1.0187
[torch.DoubleTensor of size 1x4709]

Variable containing:
 188
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.4116 -3.3545 -0.5964  ...   1.9018 -0.9374 -1.2495
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4628
[torch.LongTensor of size 1]

Variable containing:
-3.0072e-03 -3.9732e-02 -5.5244e-03  ...   2.0421e-02 -6.0010e-03 -1.1921e-02
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2238
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.1941 -3.4189 -1.0228  ...   1.9221 -0.7530 -1.5904
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4143
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.0305 -3.8312 -0.8215  ...   2.1879 -0.

Variable containing:
1.00000e-02 *
-0.4983 -3.8784 -1.0366  ...   1.8670 -0.5667 -1.0702
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4145
[torch.LongTensor of size 1]

Variable containing:
-5.4854e-03 -3.6275e-02 -1.2386e-02  ...   1.5870e-02 -5.4456e-03 -1.1081e-02
[torch.DoubleTensor of size 1x4709]

Variable containing:
 205
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.9564 -3.8988 -0.9944  ...   1.5321 -0.5983 -0.6767
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4625
[torch.LongTensor of size 1]

Variable containing:
-3.6247e-03 -3.4390e-02 -1.1541e-02  ...   1.8281e-02 -6.7412e-03 -9.4143e-03
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2253
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.1575 -3.1783 -1.4166  ...   1.9241 -0.8022 -1.4949
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4610
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.0780 -3.6134 -0.6917  ...   

Variable containing:
1.00000e-02 *
-0.6461 -3.2291 -1.1715  ...   2.1971 -1.0031 -0.6119
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4143
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.4802 -3.7288 -0.8931  ...   2.2485 -0.9369 -0.6590
[torch.DoubleTensor of size 1x4709]

Variable containing:
 192
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3868 -3.7490 -0.8607  ...   2.2315 -0.7457 -0.9116
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4628
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3740 -4.1500 -0.7185  ...   2.1059 -0.5316 -1.0023
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4142
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.6259 -3.4822 -0.8444  ...   2.0551 -0.3499 -1.0956
[torch.DoubleTensor of size 1x4709]

Variable containing:
 195
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2262 -3.0948 -0.8403  ...   1.4814 -1.1300 -0.959

...

Variable containing:
1.00000e-02 *
-0.0479 -3.9056 -0.6974  ...   1.9027 -0.4252 -0.9510
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4627
[torch.LongTensor of size 1]

Variable containing:
-2.5388e-03 -3.3150e-02 -1.0588e-02  ...   2.2305e-02 -8.0335e-03 -9.7136e-03
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2235
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3011 -3.1654 -0.9384  ...   1.8597 -1.1884 -1.4818
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4138
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.0854 -3.4183 -0.9375  ...   1.7056 -1.1670 -1.6051
[torch.DoubleTensor of size 1x4709]

Variable containing:
 188
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.1849 -3.3191 -0.6339  ...   1.7575 -1.1989 -1.5638
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4627
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2936 -3.0379 -1.1429  ...   2.2689 -1.

Variable containing:
1.00000e-02 *
-0.3220 -4.1302 -0.8266  ...   2.3571 -0.1590 -0.5557
[torch.DoubleTensor of size 1x4709]

Variable containing:
 181
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2282 -3.9941 -0.7227  ...   2.1967 -0.3812 -0.9627
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4628
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2249 -4.3307 -0.6447  ...   2.0956 -0.3840 -0.9963
[torch.DoubleTensor of size 1x4709]

Variable containing:
 183
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3454 -3.9760 -0.8429  ...   1.7608 -0.5198 -0.7504
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4629
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.0460 -4.0958 -0.9519  ...   1.8723 -0.5447 -0.5422
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2231
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.1426 -3.4347 -1.0941  ...   1.5691 -0.2397 -0.896

Variable containing:
1.00000e-02 *
 0.0153 -3.4419 -1.0975  ...   1.7521 -0.6913 -1.1040
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4627
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.1674 -3.1500 -1.3425  ...   2.2627 -0.9944 -0.8922
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2231
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.0667 -2.9889 -1.3899  ...   1.7817 -0.4582 -1.0969
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4139
[torch.LongTensor of size 1]

Variable containing:
-1.6890e-03 -3.4854e-02 -8.0265e-03  ...   1.6578e-02 -5.5185e-03 -9.7028e-03
[torch.DoubleTensor of size 1x4709]

Variable containing:
 188
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2776 -3.4037 -0.4651  ...   1.7604 -0.7238 -1.3332
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4628
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.2494 -3.1238 -1.0000  ...   1.8114 -0.

Variable containing:
1.00000e-02 *
 0.3314 -3.7144 -0.9317  ...   1.8048 -0.3989 -0.8787
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4625
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
 0.3414 -3.3086 -1.0929  ...   1.8326 -0.4822 -0.9456
[torch.DoubleTensor of size 1x4709]

Variable containing:
 2241
[torch.LongTensor of size 1]

Variable containing:
-1.4529e-03 -3.4933e-02 -8.2287e-03  ...   1.8732e-02 -5.1946e-03 -1.0791e-02
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4610
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.0440 -3.7639 -0.3159  ...   1.6375 -0.5425 -0.8740
[torch.DoubleTensor of size 1x4709]

Variable containing:
 4140
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.2947 -3.8441 -0.4867  ...   1.8579 -0.4066 -1.2166
[torch.DoubleTensor of size 1x4709]

Variable containing:
 183
[torch.LongTensor of size 1]

Variable containing:
1.00000e-02 *
-0.3697 -3.6686 -0.8127  ...   1.7253 -0.

KeyboardInterrupt: 